In [1]:
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont

def create_video_with_typewriter_effect(text, output_filename, fontsize=40, video_fps=10, video_duration=10):
    font = ImageFont.truetype("arial.ttf", fontsize)
 
    video_width = 1080
    video_height = 1920
    video_size = (video_width, video_height)
 
    video = cv2.VideoWriter(output_filename, 
                            cv2.VideoWriter_fourcc(*'mp4v'), 
                            video_fps, 
                            video_size)

   
    text_width, text_height = font.getbbox(text)[2:4]   
 
    text_fade_duration = 1 
    text_typing_duration = video_duration - text_fade_duration
 
    num_typing_frames = int(text_typing_duration * video_fps)
    num_fade_frames = int(text_fade_duration * video_fps)
 
    for frame_idx in range(num_typing_frames):
        frame = np.zeros((video_height, video_width, 3), dtype=np.uint8)
        frame_pil = Image.fromarray(frame)
        draw = ImageDraw.Draw(frame_pil)

        current_text = text[:int(len(text) * frame_idx / num_typing_frames)]
        draw.text(((video_width - text_width) // 2, (video_height - text_height) // 2), 
                  current_text, font=font, fill=(255, 255, 255))

        video.write(np.array(frame_pil))
 
    for frame_idx in range(num_fade_frames):
        frame = np.zeros((video_height, video_width, 3), dtype=np.uint8)
        frame_pil = Image.fromarray(frame)
        draw = ImageDraw.Draw(frame_pil)

        alpha = 255 - int(255 * frame_idx / num_fade_frames)
        draw.text(((video_width - text_width) // 2, (video_height - text_height) // 2), 
                  text, font=font, fill=(255, 255, 255))

        video.write(np.array(frame_pil))

    video.release()
    print(f"Video saved as {output_filename}")
 
text = "This is a typewriter animation crea ."
output_filename = "typewriter_animation1.mp4"
create_video_with_typewriter_effect(text, output_filename)


Video saved as typewriter_animation1.mp4


In [2]:
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont

def split_text_into_segments(text, font, max_width):
    """Split the text into segments that fit within max_width."""
    words = text.split()
    segments = []
    current_segment = ""
    for word in words:
        test_line = current_segment + (" " if current_segment else "") + word
        # Get width of test_line
        w = font.getbbox(test_line)[2]
        if w <= max_width:
            current_segment = test_line
        else:
            if current_segment:
                segments.append(current_segment)
            current_segment = word
    if current_segment:
        segments.append(current_segment)
    return segments

def create_video_with_typewriter_effect(scene_data, output_filename, fontsize=60, video_fps=30):
    # Load the font
    font = ImageFont.truetype("arial.ttf", fontsize)
    
    # Set video size for YouTube Shorts (1080x1920)
    video_width = 1080
    video_height = 1920
    video_size = (video_width, video_height)
    
    # Create video writer
    video = cv2.VideoWriter(output_filename,
                            cv2.VideoWriter_fourcc(*'mp4v'),
                            video_fps,
                            video_size)
    
    for scene in scene_data:
        narration = scene['narration']
        scene_duration = scene['audio_duration']  # in seconds
        
        # Split narration into segments that fit on one line (based on available width)
        segments = split_text_into_segments(narration, font, video_width - 40)  # 40 pixels margin
        num_segments = len(segments)
        if num_segments == 0:
            continue
        
        # Allocate equal duration for each segment
        segment_duration = scene_duration / num_segments
        segment_frames = int(segment_duration * video_fps)
        
        for segment in segments:
            total_chars = len(segment)
            for frame_idx in range(segment_frames):
                frame = np.zeros((video_height, video_width, 3), dtype=np.uint8)
                frame_pil = Image.fromarray(frame)
                draw = ImageDraw.Draw(frame_pil)
                
                # Calculate how many characters to show (typewriter effect)
                num_chars = int(frame_idx * total_chars / segment_frames)
                text_to_display = segment[:num_chars]
                
                # Measure text size and center it
                text_width, text_height = font.getbbox(text_to_display)[2:4]
                x = (video_width - text_width) // 2
                y = (video_height - text_height) // 2
                
                draw.text((x, y), text_to_display, font=font, fill=(255, 255, 255))
                video.write(np.array(frame_pil))
    
    video.release()
    print(f"Video saved as {output_filename}")

# Example usage with scene data
scene_data = [
    {'narration': "Once upon a time, in a small village, lived a poor but clever man named Mircho. He lived with his mother in a small hut. Once upon a time, in a small village, lived a poor but clever man named Mircho. He lived with his mother in a small hut.", 'audio_duration': 8},

]
output_filename = "typewriter_animation2.mp4"
create_video_with_typewriter_effect(scene_data, output_filename)


Video saved as typewriter_animation2.mp4


In [3]:
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont

def create_video_with_typewriter_effect(scene_data, output_filename, fontsize=60, video_fps=30):
    font = ImageFont.truetype("arial.ttf", fontsize)
    
    # Set video size for YouTube Shorts (1080x1920)
    video_width = 1080
    video_height = 1920
    video_size = (video_width, video_height)
    
    # Create video writer
    video = cv2.VideoWriter(output_filename, 
                            cv2.VideoWriter_fourcc(*'mp4v'), 
                            video_fps, 
                            video_size)
    
    for scene in scene_data:
        narration = scene['narration']
        scene_duration = scene['audio_duration']  # Ensure audio duration is available
        num_frames = int(scene_duration * video_fps)
        
        # Split the narration into individual lines
        lines = narration.splitlines()
        
        # Variable to hold the current progress of typing each line
        current_line = ""
        
        for line in lines:
            for frame_idx in range(num_frames):
                frame = np.zeros((video_height, video_width, 3), dtype=np.uint8)
                frame_pil = Image.fromarray(frame)
                draw = ImageDraw.Draw(frame_pil)
                
                # Calculate how many characters to show for the current frame
                num_chars = int(frame_idx * len(line) / num_frames)
                current_line = line[:num_chars]
                
                # Draw the text on the center of the screen
                text_width, text_height = font.getbbox(current_line)[2:4]
                y_offset = video_height // 2 - text_height // 2
                draw.text(((video_width - text_width) // 2, y_offset), 
                          current_line, font=font, fill=(255, 255, 255))
                
                # Write the frame to the video
                video.write(np.array(frame_pil))
    
    video.release()
    print(f"Video saved as {output_filename}")

# Example usage with scene data
scene_data = [
    {'narration': "Once upon a time, in a small village, lived a poor but clever man named Mircho.", 'audio_duration': 5},
    {'narration': "One day, the king announced a contest: solve three impossible tasks, win riches!", 'audio_duration': 4}
]
output_filename = "typewriter_animation3.mp4"
create_video_with_typewriter_effect(scene_data, output_filename)


Video saved as typewriter_animation3.mp4


In [4]:
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont

def create_video_with_typewriter_effect(scene_data, output_filename, fontsize=60, video_fps=30):
    font = ImageFont.truetype("arial.ttf", fontsize)
    
    # Set video size for YouTube Shorts (1080x1920)
    video_width = 1080
    video_height = 1920
    video_size = (video_width, video_height)
    
    # Create video writer
    video = cv2.VideoWriter(output_filename, 
                            cv2.VideoWriter_fourcc(*'mp4v'), 
                            video_fps, 
                            video_size)
    
    for scene in scene_data:
        narration = scene['narration']
        scene_duration = scene['audio_duration']  # Ensure audio duration is available
        num_frames = int(scene_duration * video_fps)
        
        # Create a typewriter effect for each frame
        text_to_display = ""
        for frame_idx in range(num_frames):
            frame = np.zeros((video_height, video_width, 3), dtype=np.uint8)
            frame_pil = Image.fromarray(frame)
            draw = ImageDraw.Draw(frame_pil)
            
            # Determine how much text to display for this frame
            num_chars = int(frame_idx * len(narration) / num_frames)
            text_to_display = narration[:num_chars]
            
            # Break text into lines, ensuring no line exceeds screen width
            lines = []
            current_line = ""
            words = text_to_display.split()
            for word in words:
                # Add the word to the current line if it fits, otherwise start a new line
                if font.getbbox(current_line + " " + word)[2] < video_width:
                    current_line += " " + word
                else:
                    lines.append(current_line.strip())
                    current_line = word
            if current_line:
                lines.append(current_line.strip())
            
            # Scroll lines if they exceed the screen height
            y_offset = video_height // 2 - (len(lines) * fontsize) // 2
            for line in lines:
                text_width, text_height = font.getbbox(line)[2:4]
                draw.text(((video_width - text_width) // 2, y_offset), 
                          line, font=font, fill=(255, 255, 255))
                y_offset += fontsize + 10  # Space between lines
            
            # Write frame to video
            video.write(np.array(frame_pil))
    
    video.release()
    print(f"Video saved as {output_filename}")

# Example usage with scene data
scene_data = [
    {'narration': "Once upon a time, in a small village, lived a poor but clever man named Mircho. He lived with his mother in a small hut.", 'audio_duration': 5},
    {'narration': "One day, the king announced a contest: solve three impossible tasks, win riches!", 'audio_duration': 4}
]
output_filename = "typewriter_animation5.mp4"
create_video_with_typewriter_effect(scene_data, output_filename)


Video saved as typewriter_animation5.mp4
